#### Depencies

In [26]:
import geopy
import openml
import pandas as pd
import geopandas as gpd
import glob

from geopy.geocoders import Nominatim
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from tqdm import tqdm
import os
import numpy as np

from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import re

# Data Profilling

## **An Exploratory Data Analysis of U.S Individual Donations for Political Campaigs**


---
A Comissão Federal de Eleições (FEC) é uma agência independente que visa mediar e regular o financiamente de campanhas presidencias dos EUA. Sob o pretexto de registrar cada doação dos cidadãos, criou-se a base que será aqui trabalhada. A partir das bibliotecas **Vega-Altair** e **Plotly**, explora-se a criação de visualizações para compreender e investigar o comportamento das contribuições políticas ao redor do país.


---



### **As variáveis da base**

#### Variáveis Preditivas

* **cmte_id**: ID do comite para o qual a contribuição foi feita. Cada comitê possui um identificador único. Neste dataset estão presentes 7112 comites diferentes.

* **amndt_ind**: Indicador de Emenda. Tal variável é usada para mostrar se uma transação ou um registro é uma emenda a um registro anteriormente apresentado, sendo classificada por uma das três opções:
  * N - Novo registro;

  * A - Emenda a um registro anterior;
  * C - Correção/alteração num registro.

* **rpt_tp**: Tipo de Relatório de contribuição do Comitê ao qual o contribuinte está associado. Seus tipos são comumente diferenciados pela sua recorrência(mensal, trimestral, pré-eleição e etc).

* **transaction_pgi**: Indicador referente a qual etapa do ciclo eleitoral a contribuição em questão está associada.
  *  G - General Election, fase final do ciclo, onde um candidato é eleito para o cargo
  *  P - Primary Election, refere-se às eleições que são internas aos partidos
  *  S - Special Election, eleições convocadas fora do ciclo eleitoral regular para preencher vagas inesperadas
  *  E - Election,
  *  C - Convenction, Convenções Partidárias, que podem ser usadas para selecionar candidatos ou tomar outras decisões internas do partido.
  * O - Other,

  * R - Runoff Election (Segundo Turno)

  * 0 - ??

* **image_num**: Identificador único associado à imagem digitalizada do relatório financeiro.

* **transaction_tp**
  * 10 - Contribuição Financeira direta de um indivíduo ou entidade para um comitê de campanha de um candidato
  * 11 - Contribuição de uma Tribo Nativa
  * 15 - Contribuição financeira direta feita por um indivíduo, sociedade ou LLC (Empresa de responsabilidade limitada) para comitês políticos tradicionais, excluindo Super PACs e Hybrid PACs.
  * 15C - Contribuição de candidatos
  * 15E - Contribuições Direcionadas feitas por indivíduos, sociedades ou LLC para comitês políticos tradicionais
  * 19 - Doação para comunicação eleitoral

  * 20Y - Reembolso de Fundos não Eleitorais. Devolução de fundos que foram originalmente atribuídos a atividades não eleitorais.
  * 21Y - Reembolso de uma Tribo Nativa
  * 22Y - Reembolso de uma contribuição de uma indivíduo, parceiros ou uma empresa de responsabilidade limitada
  * 24I - Contribuição Direcionada por cheque
  * 24T - Contribuição Direcionada utilizando fundos do tesouro do comitê intermediário

* **entity_tp**
  * IND - Individual
  * ORG - Organização
  * CCM - Comitê do Candidato
  * PAC - Comitê de Ação Política
  * CAN - Candidato
  * COM - Comitê

  * PTY - Comitê Oficial do Partido

* **name** - Nome do Contribuinte
* **city** - Cidade do Contribuinte

* **state** - Estado do Contribuinte

* **zip_code** - Código Postal do Contribuinte

* **employer** - Empregador do Contribuinte

* **ocuppation** - Emprego do Contribuinte

* **transaction_dt** - Data de transação das contribuições

* **other_id** - ID de identificação de uma terceira parte envolvida na transação

* **tran_id** - ID único da transação

* **file_num** - Número do arquivo

* **memo_cd** - Identificador associado a presença de um memorando na transação, o qual fornece um contexto adicional à transação

* **memo_text** - Texto do memorando

*  **sub_id** - Identificador de submissão

#### Variável Alvo
* **transaction_amt**: O valor da transação em dólares. Indica a quantidade de dinheiro envolvida em uma contrbuição individual para campanhas políticas.

Assim, o dataset é composto por 5 variáveis de Identificação única, 13 variáveis categóricas, uma variável de texto, uma variável de data e uma varável numérica (alvo)



In [16]:
dataset = openml.datasets.get_dataset(42080)
df, _, _, _ = dataset.get_data(dataset_format='dataframe')
path_shapefile = "cb_2018_us_state_500k\cb_2018_us_state_500k.shp"

state_boundaries = gpd.read_file(path_shapefile, columns=['STUSPS', 'NAME', 'geometry'])
state_boundaries.rename(columns={'STUSPS': 'state',
                           'Name': 'state_name'}, inplace=True)

df_EUA = state_boundaries.merge(df, on='state', how='inner')

In [17]:
df.shape

(3348209, 21)

In [18]:
# Número de categorias únicas das variáveis categóricas
categorical_cols = df.select_dtypes(include='object')
num_categories = categorical_cols.nunique().sort_values(ascending=False)
print(num_categories)

tran_id            2999272
name               1534998
employer            657449
occupation          145304
zip_code             86334
city                 27709
cmte_id               7112
other_id              2543
state                   67
rpt_tp                  26
transaction_tp          11
transaction_pgi          8
entity_tp                7
amndt_ind                3
memo_cd                  2
dtype: int64


# Preprocessing

In [5]:
#Percentual de Dados Faltantes
missing_percent = df.isna().sum().sort_values(ascending=False) / df.shape[0] * 100
missing_percent = missing_percent[missing_percent > 0]
print(missing_percent)

memo_cd            97.314325
other_id           92.992313
memo_text          87.438747
transaction_pgi    28.104966
employer           10.371216
occupation          5.546966
state               0.258556
city                0.064990
entity_tp           0.042948
transaction_dt      0.010573
tran_id             0.009766
name                0.004211
file_num            0.000030
dtype: float64


In [6]:
#Co-ocorrência das variáveis com valores faltantes significantes
significant_missing_columns = ['occupation', 'employer', 'transaction_pgi', 'memo_cd', 'memo_text', 'other_id']
co_occur = df[significant_missing_columns].isna().astype(int)
co_occur_agg = co_occur.groupby(significant_missing_columns).size().reset_index(name='count').sort_values(by='count', ascending=False)
co_occur_agg.head(15)

,occupation,employer,transaction_pgi,memo_cd,memo_text,other_id,count
7,0,0,0,1,1,1,1890857
15,0,0,1,1,1,1,683124
54,1,1,0,1,1,1,140372
5,0,0,0,1,0,1,93993
23,0,1,0,1,1,1,88439
4,0,0,0,1,0,0,85737
12,0,0,1,1,0,0,80218
13,0,0,1,1,0,1,60084
31,0,1,1,1,1,1,52315
1,0,0,0,0,0,1,47905


In [7]:
#Co-ocorrência das variáveis com dados faltantes que podem ser imputadas
significant_missing_columns = ['occupation', 'employer']
co_occur = df[significant_missing_columns].isna().astype(int)
co_occur_agg = co_occur.groupby(significant_missing_columns).size().reset_index(name='count').sort_values(by='count', ascending=False)
co_occur_agg.head(15)

,occupation,employer,count
0,0,0,2994621
3,1,1,179386
1,0,1,167864
2,1,0,6338


In [19]:
# Exclui-se as variáveis de indentificadores únicos
df.drop(columns=['name', 'sub_id', 'tran_id', 'image_num'], inplace=True)

# Exclui-se as variáveis 'memo_cd', 'other_id', 'memo_text'
df.drop(columns=['memo_cd', 'other_id', 'memo_text'], inplace=True)

colunas_com_poucos_dados_faltantes = ["state", "city", "entity_tp", "transaction_dt", "file_num"]
df = df.dropna(subset=colunas_com_poucos_dados_faltantes)
df['transaction_pgi'] = df['transaction_pgi'].fillna("Desconhecido")

In [20]:
df['transaction_dt'] = df['transaction_dt'].astype(int).astype(str)

# Garantir que todos os valores tenham 8 dígitos (ex.: '9122012' -> '09122012')
df['transaction_dt'] = df['transaction_dt'].str.zfill(8)

# Criar uma coluna datetime usando pd.to_datetime
df['transaction_dt'] = pd.to_datetime(df['transaction_dt'], format='%m%d%Y')

### Data imputation with word embedding

In [27]:
sample_fraction = 0.01  # 1% dos dados
np.random.seed(42)  # Para reprodutibilidade

# Selecionar uma amostra de 1% dos dados
sampled_df = df.sample(frac=sample_fraction, random_state=42)

In [28]:
def preprocess_text(text):
    # Handle missing values
    if pd.isnull(text):
        return 'missing'
    # Convert to lowercase
    text = text.lower()
    # Remove special characters and punctuation
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Remove common suffixes and stopwords
    text = re.sub(r'\b(inc|llc|corp|corporation|co|ltd|and|the)\b', '', text)
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text


for col in ['employer', 'occupation']:
    sampled_df[col] = sampled_df[col].apply(preprocess_text)

In [29]:
# Garantir que a coluna employer e occupation tenha valores únicos na amostra
unique_employers = sampled_df[sampled_df['employer'] != 'missing']['employer'].unique()
unique_occupations = sampled_df[sampled_df['occupation'] != 'missing']['occupation'].unique()

# Carregar o modelo
model = SentenceTransformer('all-MiniLM-L6-v2')

# Gerar embeddings para employers
employer_embeddings = {}
print("Generating embeddings for employers from the sample...")
for employer in tqdm(unique_employers, desc="Employers Progress", unit="employer"):
    embedding = model.encode(employer)
    employer_embeddings[employer] = embedding

# Gerar embeddings para occupations
occupation_embeddings = {}
print("\nGenerating embeddings for occupations from the sample...")
for occupation in tqdm(unique_occupations, desc="Occupations Progress", unit="occupation"):
    embedding = model.encode(occupation)
    occupation_embeddings[occupation] = embedding

# Função para imputar os valores ausentes
def impute_missing_values(row, embeddings_dict, column_name, model, fallback_value):
    if row[column_name] == 'missing':
        other_column = 'occupation' if column_name == 'employer' else 'employer'
        context = row[other_column]
        if context != 'missing':
            context_embedding = model.encode(context)
            similarities = cosine_similarity([context_embedding], list(embeddings_dict.values()))[0]
            max_similarity = np.max(similarities)
            if max_similarity >= 0.7:
                most_similar_index = np.argmax(similarities)
                return list(embeddings_dict.keys())[most_similar_index]
    return fallback_value

# Imputar os valores ausentes na amostra
print("\nImputing missing values...")
sampled_df['employer'] = sampled_df.apply(
    lambda row: impute_missing_values(
        row, employer_embeddings, 'employer', model, sampled_df['employer'].mode()[0]
    ),
    axis=1
)

sampled_df['occupation'] = sampled_df.apply(
    lambda row: impute_missing_values(
        row, occupation_embeddings, 'occupation', model, sampled_df['occupation'].mode()[0]
    ),
    axis=1
)

Generating embeddings for employers from the sample...


Employers Progress: 100%|██████████| 15343/15343 [03:02<00:00, 84.00employer/s] 



Generating embeddings for occupations from the sample...


Occupations Progress: 100%|██████████| 5165/5165 [00:58<00:00, 88.16occupation/s] 



Imputing missing values...


In [32]:
# Validar distribuições após a imputação
print(sampled_df['employer'].isna().sum())
print(sampled_df['occupation'].isna().sum())

0
0


In [33]:
sampled_df.to_csv("amostra.csv")

## Geocoding from zip codes

In [6]:
geolocator = Nominatim(user_agent="zip_code_locator")

# Cache para armazenar os resultados de ZIP codes já consultados
cache = {}

def get_location_from_zip(zip_code, max_retries=5):
    if zip_code in cache:
        print(f"zip code: {zip_code} já processado.")
        return zip_code, *cache[zip_code]
    
    # Implementar retentativas
    for attempt in range(max_retries):
        try:
            location = geolocator.geocode({"postalcode": zip_code, "country": "USA"})
            if location:
                result = (location.latitude, location.longitude, location.address.split(',')[1].strip())
                cache[zip_code] = result
                time.sleep(1.5)  # Reduz carga no serviço gratuito
                return zip_code, *result
        except Exception as e:
            print(f"Erro na tentativa {attempt + 1} para o ZIP code {zip_code}: {e}")
            time.sleep(2 ** attempt)  # Espera progressiva (backoff exponencial)
    
    # Retornar None se todas as tentativas falharem
    return zip_code, None, None, None


In [ ]:
unique_zip_codes = df['zip_code'].unique()

chunk_size = 2000
output_dir = "geoprocessed_chunks"
os.makedirs(output_dir, exist_ok=True)

# Verificar quais chunks já foram processados
processed_chunks = {int(file.split('_')[-1].split('.')[0]) for file in os.listdir(output_dir) if file.endswith(".csv")}

for i in range(0, len(unique_zip_codes), chunk_size):
    chunk_index = i // chunk_size + 1  

    if chunk_index in processed_chunks:
        print(f"Chunk {chunk_index} já processado. Pulando...")
        continue

    chunk = unique_zip_codes[i:i+chunk_size]
    
    results = []
    for zip_code in tqdm(chunk, desc=f"Processando chunk {chunk_index}"):
        results.append(get_location_from_zip(zip_code))
    
    chunk_df = pd.DataFrame(results, columns=['zip_code', 'latitude', 'longitude', 'cidade'])
    
    chunk_file_name = os.path.join(output_dir, f"processed_zip_codes_chunk_{chunk_index}.csv")
    chunk_df.to_csv(chunk_file_name, index=False)
    print(f"Chunk {chunk_index} salvo com sucesso em {chunk_file_name}")

Chunk 1 já processado. Pulando...
Chunk 2 já processado. Pulando...
Chunk 3 já processado. Pulando...
Chunk 4 já processado. Pulando...
Chunk 5 já processado. Pulando...


Processando chunk 6: 100%|██████████| 2000/2000 [1:23:49<00:00,  2.51s/it]


Chunk 6 salvo com sucesso em geoprocessed_chunks\processed_zip_codes_chunk_6.csv


Processando chunk 7: 100%|██████████| 2000/2000 [1:23:32<00:00,  2.51s/it]


Chunk 7 salvo com sucesso em geoprocessed_chunks\processed_zip_codes_chunk_7.csv


...

Chunk 41 salvo com sucesso em geoprocessed_chunks\processed_zip_codes_chunk_41.csv


Processando chunk 42: 100%|██████████| 2000/2000 [1:21:37<00:00,  2.45s/it]


Chunk 42 salvo com sucesso em geoprocessed_chunks\processed_zip_codes_chunk_42.csv


Processando chunk 43: 100%|██████████| 1802/1802 [1:15:07<00:00,  2.50s/it]

Chunk 43 salvo com sucesso em geoprocessed_chunks\processed_zip_codes_chunk_43.csv


## Merge final

In [12]:
output_dir = "geoprocessed_chunks"

all_chunk_files = glob.glob(os.path.join(output_dir, "processed_zip_codes_chunk_*.csv"))
processed_chunks_df = pd.concat([pd.read_csv(file) for file in all_chunk_files], ignore_index=True)
processed_chunks_df.shape

(85802, 4)

In [13]:
processed_chunks_df

,zip_code,latitude,longitude,cidade
0,28791.0,35.348586,-82.502214,Henderson County
1,39475.0,31.162820,-89.414966,Lamar County
2,61834.0,40.158154,-87.627952,Danville
3,75462.0,33.668733,-95.493766,Paris
4,24740.0,37.369439,-81.096546,Princeton
...,...,...,...,...
85797,7505.0,40.916434,-74.171520,Paterson
85798,7843.0,40.938197,-74.660831,Hopatcong
85799,10925.0,41.211813,-74.301091,Village of Greenwood Lake
85800,7035.0,40.918130,-74.309320,Lincoln Park


In [15]:
merge_1 = processed_chunks_df[['zip_code', 'latitude', 'longitude']]
amostra_teste = pd.merge(merge_1, df, on='zip_code', how='inner')
amostra_teste.shape

(121979, 16)

In [16]:
amostra_teste.isna().sum()

zip_code               0
latitude           12326
longitude          12326
cmte_id                0
amndt_ind              0
rpt_tp                 0
transaction_pgi        0
transaction_tp         0
entity_tp              0
city                   0
state                  0
employer           12498
occupation          6905
transaction_dt         0
transaction_amt        0
file_num               0
dtype: int64

In [ ]:
amostra_teste.to_csv("amostra_coordenadas.csv")